In [16]:
!git clone https://github.com/misbah4064/human-pose-estimation-opencv.git
%cd human-pose-estimation-opencv/

Cloning into 'human-pose-estimation-opencv'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 20 (delta 5), reused 5 (delta 5), pack-reused 12 (from 1)
Receiving objects: 100% (20/20), 10.09 MiB | 7.76 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/home/naveed/Documents/Nexpred Solutions/Month4_September/Week 1/Day 1/human-pose-estimation-opencv/human-pose-estimation-opencv


/home/naveed/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [17]:
import cv2 as cv
import numpy as np

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

width = 368
height = 368
inWidth = width
inHeight = height

net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
thr = 0.2

def poseDetector(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()

    return frame

In [18]:
import cv2
cap = cv2.VideoCapture('Test.mp4')
ret, frame = cap.read()
frame_height, frame_width, _ = frame.shape
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
print("Processing Video...")
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    out.release()
    break
  output = poseDetector(frame)
  out.write(output)
out.release()
print("Done processing video")

Processing Video...
Done processing video


### testing on pipeline

In [19]:
import datetime
import cv2 as cv
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# YOLO model and DeepSort initialization
model = YOLO("yolov8n.pt")
tracker = DeepSort(max_age=50)

# Body parts and pose pairs for OpenPose
BODY_PARTS = {"Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
              "LEye": 15, "REar": 16, "LEar": 17, "Background": 18}

POSE_PAIRS = [["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
              ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
              ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
              ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
              ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]]

# Load OpenPose model
try:
    net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
    print("OpenPose model loaded successfully.")
except Exception as e:
    print("Failed to load OpenPose model:", e)

inWidth, inHeight = 368, 368
thr = 0.2

def poseDetector(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # Only need the first 19 elements for pose

    points = []
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    return frame

def create_video_writer(video_cap, output_filename):
    frame_width = int(video_cap.get(cv.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv.CAP_PROP_FPS))
    fourcc = cv.VideoWriter_fourcc(*'MP4V')
    writer = cv.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))
    return writer

# Initialize video capture and writer
video_cap = cv.VideoCapture("Test.mp4")
if not video_cap.isOpened():
    print("Error: Unable to open video file.")
else:
    print("Video file loaded successfully.")

writer = create_video_writer(video_cap, "Testoutput1.mp4")

while True:
    start = datetime.datetime.now()
    ret, frame = video_cap.read()
    if not ret:
        print("No more frames to read or video file error.")
        break

    # Run YOLO model for detection
    detections = model(frame)[0]

    if detections is None or len(detections) == 0:
        print("No detections from YOLO.")
        continue

    results = []
    for data in detections.boxes.data.tolist():
        confidence = data[4]
        if float(confidence) < 0.2:
            continue
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        class_id = int(data[5])
        results.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

    # Update DeepSort tracker with detections
    tracks = tracker.update_tracks(results, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        xmin, ymin, xmax, ymax = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])

        # Crop image of tracked person
        cropped_image = frame[ymin:ymax, xmin:xmax]
        os.makedirs(f"tracked_images/{track_id}", exist_ok=True)
        cv.imshow(f"tracked_images/{track_id}/frame_{start.timestamp()}.jpg", cropped_image)

        # Perform pose estimation
        pose_frame = poseDetector(cropped_image)

        # Overlay pose on original frame
        frame[ymin:ymax, xmin:xmax] = pose_frame

        # Draw bounding box and track ID
        cv.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv.rectangle(frame, (xmin, ymin - 20), (xmin + 20, ymin), (0, 255, 0), -1)
        cv.putText(frame, str(track_id), (xmin + 5, ymin - 8), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Calculate and display FPS
    end = datetime.datetime.now()
    print(f"Time to process 1 frame: {(end - start).total_seconds() * 1000:.0f} milliseconds")
    fps = f"FPS: {1 / (end - start).total_seconds():.2f}"
    cv.putText(frame, fps, (50, 50), cv.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 8)

    writer.write(frame)
    if cv.waitKey(1) == ord("q"):
        break

video_cap.release()
writer.release()
cv.destroyAllWindows()

OpenPose model loaded successfully.
Video file loaded successfully.


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'



0: 384x640 35 persons, 2 birds, 113.9ms
Speed: 3.5ms preprocess, 113.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Time to process 1 frame: 5390 milliseconds

0: 384x640 34 persons, 3 birds, 109.3ms
Speed: 2.2ms preprocess, 109.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Time to process 1 frame: 1848 milliseconds

0: 384x640 35 persons, 3 birds, 105.9ms
Speed: 2.1ms preprocess, 105.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Time to process 1 frame: 8444 milliseconds

0: 384x640 36 persons, 2 birds, 105.6ms
Speed: 3.4ms preprocess, 105.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Time to process 1 frame: 8836 milliseconds

0: 384x640 37 persons, 2 birds, 128.0ms
Speed: 2.1ms preprocess, 128.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Time to process 1 frame: 8813 milliseconds

0: 384x640 37 persons, 2 birds, 109.3ms
Speed: 2.3ms preprocess, 109.3ms inference, 0.8ms

error: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:798: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
